# 5. Data Preprocessing

Disini kita bakal:
- Null Handling
- Outlier Handling

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('./dataset/train_cleaned.csv')
df_train['datetime_iso'] = pd.to_datetime(df_train['datetime_iso'])

df_test = pd.read_csv('./dataset/test_cleaned.csv')
df_test['datetime_iso'] = pd.to_datetime(df_test['datetime_iso'])

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   datetime      341880 non-null  int64              
 1   datetime_iso  341880 non-null  datetime64[ns, UTC]
 2   time-zone     341880 non-null  int64              
 3   temp          341880 non-null  float64            
 4   visibility    51112 non-null   object             
 5   d_point       341880 non-null  float64            
 6   feels         341880 non-null  float64            
 7   min_temp      341880 non-null  float64            
 8   max_temp      341880 non-null  float64            
 9   prssr         341880 non-null  float64            
 10  sea_level     192964 non-null  object             
 11  grnd_level    192919 non-null  object             
 12  hum           341880 non-null  float64            
 13  wind_spd      339654 non-null  float64      

## Drop unnecesary Feature

In [4]:
df_train = df_train.drop(['datetime','time-zone','visibility','sea_level','grnd_level','rain_3h','snow_1h','snow_3h'],axis = 1)
df_test = df_test.drop(['datetime','time-zone','visibility','sea_level','grnd_level','rain_3h','snow_1h','snow_3h'],axis = 1)

In [5]:
df_train.head()

,datetime_iso,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds
0,1979-01-01 00:00:00+00:00,24.75,23.89,25.76,24.28,25.22,1012.0,95.0,0.82,320.0,0.00,100.0
1,1979-01-01 01:00:00+00:00,24.58,23.73,25.57,23.99,25.26,1012.0,95.0,0.96,338.0,0.00,100.0
2,1979-01-01 02:00:00+00:00,26.60,24.06,26.60,26.10,27.39,1012.0,86.0,1.22,339.0,0.00,99.0
3,1979-01-01 03:00:00+00:00,27.31,24.37,30.90,26.59,28.36,1012.0,84.0,1.08,342.0,0.13,94.0
4,1979-01-01 04:00:00+00:00,27.41,25.05,31.54,26.58,28.31,1011.0,87.0,0.86,336.0,0.34,100.0


# Null Handling

## Null Handling: Median

Hanya ada satu feature yang memiliki null, yaitu wind_spd.

Oh ternyata ada d_point yang null di df_test

In [6]:
df_train['wind_spd'] = df_train['wind_spd'].interpolate(method='linear')
df_test['wind_spd'] = df_test['wind_spd'].interpolate(method='linear')
df_test['d_point'] = df_test['d_point'].interpolate(method='linear')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   datetime_iso  341880 non-null  datetime64[ns, UTC]
 1   temp          341880 non-null  float64            
 2   d_point       341880 non-null  float64            
 3   feels         341880 non-null  float64            
 4   min_temp      341880 non-null  float64            
 5   max_temp      341880 non-null  float64            
 6   prssr         341880 non-null  float64            
 7   hum           341880 non-null  float64            
 8   wind_spd      341880 non-null  float64            
 9   wind_deg      341880 non-null  float64            
 10  rain_1h       341880 non-null  float64            
 11  clouds        341880 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(11)
memory usage: 31.3 MB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   datetime_iso  49368 non-null  datetime64[ns, UTC]
 1   temp          49368 non-null  float64            
 2   d_point       49368 non-null  float64            
 3   feels         49368 non-null  float64            
 4   min_temp      49368 non-null  float64            
 5   max_temp      49368 non-null  float64            
 6   prssr         49368 non-null  float64            
 7   hum           49368 non-null  float64            
 8   wind_spd      49368 non-null  float64            
 9   wind_deg      49368 non-null  float64            
 10  clouds        49368 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(10)
memory usage: 4.1 MB


# Outlier Handling

## Outlier Handling: Interpolation

In [9]:
df_train = df_train.drop(['datetime_iso'],axis = 1)

Konsep Kerja: Ada outlier -> Dijadiin Null -> Diinterpolasi lagi hehe

In [11]:
# Function to identify outliers using 1.5 IQR rule
def identify_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (series < lower_bound) | (series > upper_bound)

# Identify rows with at least one outlier
outlier_mask = df_train.apply(identify_outliers, axis=1).any(axis=1)

# Create a new DataFrame with the rows containing outliers
new_df = df_train[outlier_mask]

# Print the new DataFrame
print(new_df)

         temp  d_point   feels  min_temp  max_temp   prssr   hum  wind_spd  \
0       24.75    23.89   25.76     24.28     25.22  1012.0  95.0      0.82   
1       24.58    23.73   25.57     23.99     25.26  1012.0  95.0      0.96   
2       26.60    24.06   26.60     26.10     27.39  1012.0  86.0      1.22   
3       27.31    24.37   30.90     26.59     28.36  1012.0  84.0      1.08   
4       27.41    25.05   31.54     26.58     28.31  1011.0  87.0      0.86   
...       ...      ...     ...       ...       ...     ...   ...       ...   
341875  25.06    24.55  131.53     24.44     25.69  1007.0  97.0      0.90   
341876  24.51    24.17   25.58     23.89     25.13  1006.0  98.0      0.85   
341877  24.63    24.29  129.32     24.00    126.96  1007.0  98.0      1.54   
341878  26.68    24.71   29.76     25.02     27.25  1008.0  89.0      1.46   
341879  26.49    24.90   26.49     26.02     27.09  1008.0  91.0      1.56   

        wind_deg  rain_1h  clouds  
0          320.0     0.00  

In [ ]:
new_df

In [12]:
#Outlier Detection and Replacement (Quantile and Median Method)

df_columns_outlier_handle = ['temp',
 'd_point',
 'feels',
 'min_temp',
 'max_temp',
 'prssr',
 'hum',
 'wind_spd',
 'wind_deg',
 'clouds']

# df_outhand = df_train.copy()

for columnName in df_columns_outlier_handle:
  #detecting outliers (1.5 below/above)
  Q1 = df_train[columnName].quantile(0.25)
  Q3 = df_train[columnName].quantile(0.75)
  # print(Q1, Q3)
  IQR = Q3-Q1
  # print(IQR)
  lwr_bound = Q1-(1.5*IQR)
  upr_bound = Q3+(1.5*IQR)

  # #replace with median 
  median = np.median(df_train[columnName])

  df_train.loc[(df_train[columnName] < lwr_bound) | (df_train[columnName] > upr_bound), columnName] = np.nan
    
    
for columnName in df_columns_outlier_handle:
  #detecting outliers (1.5 below/above)
  Q1 = df_test[columnName].quantile(0.25)
  Q3 = df_test[columnName].quantile(0.75)
  # print(Q1, Q3)
  IQR = Q3-Q1
  # print(IQR)
  lwr_bound = Q1-(1.5*IQR)
  upr_bound = Q3+(1.5*IQR)

  # #replace with median 
  median = np.median(df_test[columnName])

  df_test.loc[(df_test[columnName] < lwr_bound) | (df_test[columnName] > upr_bound), columnName] = np.nan

/tmp/ipykernel_7767/431252046.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rows_with_null.append(df_train.loc[(df_train[columnName] < lwr_bound) | (df_train[columnName] > upr_bound), columnName])
/tmp/ipykernel_7767/431252046.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rows_with_null.append(df_train.loc[(df_train[columnName] < lwr_bound) | (df_train[columnName] > upr_bound), columnName])
/tmp/ipykernel_7767/431252046.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rows_with_null.append(df_train.loc[(df_train[columnName] < lwr_bound) | (df_train[columnName] > upr_bound), columnName])
/tmp/ipykernel_7767/431252046.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   datetime_iso  341880 non-null  datetime64[ns, UTC]
 1   temp          336091 non-null  float64            
 2   d_point       331649 non-null  float64            
 3   feels         336752 non-null  float64            
 4   min_temp      336605 non-null  float64            
 5   max_temp      335463 non-null  float64            
 6   prssr         331992 non-null  float64            
 7   hum           330010 non-null  float64            
 8   wind_spd      322168 non-null  float64            
 9   wind_deg      337639 non-null  float64            
 10  rain_1h       341880 non-null  float64            
 11  clouds        239403 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(11)
memory usage: 31.3 MB


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   datetime_iso  49368 non-null  datetime64[ns, UTC]
 1   temp          49363 non-null  float64            
 2   d_point       48505 non-null  float64            
 3   feels         49367 non-null  float64            
 4   min_temp      49364 non-null  float64            
 5   max_temp      49352 non-null  float64            
 6   prssr         48784 non-null  float64            
 7   hum           48634 non-null  float64            
 8   wind_spd      47863 non-null  float64            
 9   wind_deg      49368 non-null  float64            
 10  clouds        43665 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(10)
memory usage: 4.1 MB


In [14]:
#rows_with_null = df_train[df_train.isnull().any(axis=1)]
rows_with_null.head(30)

""


### Isi elemen yang jadi null dengan interpolasi

In [12]:
for columnName in df_columns_outlier_handle:
    df_train[columnName] = df_train[columnName].interpolate(method='linear')
    df_test[columnName] = df_test[columnName].interpolate(method='linear')

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   datetime_iso  341880 non-null  datetime64[ns, UTC]
 1   temp          341880 non-null  float64            
 2   d_point       341880 non-null  float64            
 3   feels         341880 non-null  float64            
 4   min_temp      341880 non-null  float64            
 5   max_temp      341880 non-null  float64            
 6   prssr         341880 non-null  float64            
 7   hum           341880 non-null  float64            
 8   wind_spd      341880 non-null  float64            
 9   wind_deg      341880 non-null  float64            
 10  rain_1h       341880 non-null  float64            
 11  clouds        341880 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(11)
memory usage: 31.3 MB


In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   datetime_iso  49368 non-null  datetime64[ns, UTC]
 1   temp          49368 non-null  float64            
 2   d_point       49368 non-null  float64            
 3   feels         49368 non-null  float64            
 4   min_temp      49368 non-null  float64            
 5   max_temp      49368 non-null  float64            
 6   prssr         49368 non-null  float64            
 7   hum           49368 non-null  float64            
 8   wind_spd      49368 non-null  float64            
 9   wind_deg      49368 non-null  float64            
 10  clouds        49368 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(10)
memory usage: 4.1 MB


### Save in csv

In [15]:
df_train.to_csv('./dataset/train_preprocessed.csv', index=False)
df_test.to_csv('./dataset/test_preprocessed.csv', index=False)